<a href="https://colab.research.google.com/github/iiVSX/SWE3032_41_team10/blob/main/New3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.1 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.xlsx to Dataset.xlsx


In [3]:
import pandas as pd

# 데이터셋 불러오기
data = pd.read_excel('Dataset.xlsx')

In [4]:
happy_data = data[data['감정'] == '행복']
sad_data = data[data['감정'] == '슬픔']
fear_data = data[data['감정'] == '공포']
neutral_data = data[data['감정'] == '중립']
anger_data = data[data['감정'] == '분노']
aversion_data = data[data['감정'] == '혐오']
surprise_data = data[data['감정'] == '놀람']

print(len(happy_data))
print(len(sad_data))
print(len(fear_data))
print(len(neutral_data))
print(len(anger_data))
print(len(aversion_data))
print(len(surprise_data))

11615
21239
9697
51900
20934
10308
12519


In [5]:
from sklearn.utils import resample

minority_data_size = 500

happy_data_resampled = resample(happy_data, replace=False, n_samples=minority_data_size, random_state=42)
sad_data_resampled = resample(sad_data, replace=False, n_samples=minority_data_size, random_state=42)
fear_data_resampled = resample(fear_data, replace=False, n_samples=minority_data_size, random_state=42)
neutral_data_resampled = resample(neutral_data, replace=False, n_samples=minority_data_size, random_state=42)
anger_data_resampled = resample(anger_data, replace=False, n_samples=minority_data_size, random_state=42)
aversion_data_resampled = resample(aversion_data, replace=False, n_samples=minority_data_size, random_state=42)
surprise_data_resampled = resample(surprise_data, replace=False, n_samples=minority_data_size, random_state=42)

data = pd.concat([happy_data_resampled, sad_data_resampled, fear_data_resampled, neutral_data_resampled, anger_data_resampled, aversion_data_resampled, surprise_data_resampled])

print(data)

                                                       문장  감정
88040                      끝까지 한번 가보자끝까지해 ~~!!!!!!ㅋㅋㅋㅋㅋㅋㅋ  행복
121949      사람들한테 내가 이벤트에 당첨 돼서 갖고 싶었던 향수를 선물로 받았다고 자랑했어.  행복
122817  내가 10km 마라톤 대회에 출전했는데 이전의 내 기록을 훨씬 갱신했어. 너무 기분...  행복
85385                                        과학자들 존경스럽군ㅎㅎ  행복
83696                                   좋은기사..한참동안 읽었네요^^  행복
...                                                   ...  ..
64149                                            송지효는 의외다  놀람
37199                                   뭐야? 너 지금 그게 정말이야?  놀람
124956             해피도 한동안 진정을 못하더라고. 그러더니 조금 지나니 진정하더라고.  놀람
63079                       오랜만에 서울올라와서 식구들과 점심먹으러 왔었는데..  놀람
43377                                                뭐요?   놀람

[3500 rows x 2 columns]


In [6]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# 데이터셋 전처리
sentences = data['문장'].tolist()
labels = data['감정'].tolist()

# 각 문장을 토큰화하기
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 가장 긴 문장을 확인
max_len = 0
for sentence in tokenized_texts:
    max_len = max(max_len, len(sentence))
MAX_LEN = max_len

# 토큰을 인덱스로 변환하고 패딩하기
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크 생성하기
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# label 정수화
label_dict = {'행복':0, '슬픔':1, '공포':2, '중립':3, '분노':4, '혐오':5, '놀람':6}
labels = [label_dict[label] for label in labels]

In [7]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.2)

In [8]:
import numpy as np

def one_hot_encode(labels, num_classes):
    encoded_labels = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        encoded_labels[i][label] = 1
    return encoded_labels

In [9]:
num_classes = len(set(train_labels))
encoded_train_labels = one_hot_encode(train_labels, num_classes)

In [10]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW

# GPU 사용 여부
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습 데이터셋 로드
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# 데이터셋 생성
train_data = TensorDataset(train_inputs, train_masks, train_labels)

# 배치 사이즈 및 샘플링 방식 설정
batch_size = 32
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# validation 데이터셋 로드
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# 배치 사이즈 및 샘플링 방식 설정
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 모델 정의
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=7)
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)  

# 에폭 수
num_epochs = 10

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [11]:
def flat_accuracy(preds, labels):
    """
    정확도 계산 함수
    """
    preds_flat = np.argmax(preds, axis=1).flatten() # 예측값에서 가장 큰 값의 인덱스를 추출하여 1차원으로 만듦
    labels_flat = labels.flatten() # 실제값을 1차원으로 만듦
    return np.sum(preds_flat == labels_flat) / len(labels_flat)

# 모델 학습
for epoch in range(num_epochs):
    # 학습 모드로 변경
    model.train()
    
    # 손실값 초기화
    total_loss = 0
    
    # 배치별로 학습
    for step, batch in enumerate(train_dataloader):
        # 배치 데이터 GPU로 이동
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # forward 수행
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        
        # 손실값 누적
        total_loss += loss.item()
        
        # backward 수행 및 그래디언트 클리핑
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 옵티마이저 업데이트
        optimizer.step()
        
    # 평균 손실값 계산
    avg_train_loss = total_loss / len(train_dataloader)
    
    # 검증 모드로 변경
    model.eval()
    
    # 검증 데이터셋을 이용한 검증
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for batch in validation_dataloader:
        # 배치 데이터 GPU로 이동
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # 그래디언트 계산하지 않음
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            
        # 검증 손실값 누적
        total_eval_loss += loss.item()
        
        # 예측값 추출
        preds = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 정확도 계산
        total_eval_accuracy += flat_accuracy(preds, label_ids)
    
    # 평균 검증 손실값 및 정확도 계산
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # 결과 출력
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Average training loss: {avg_train_loss:.4f}")
    print(f"Average validation loss: {avg_val_loss:.4f}")
    print(f"Average validation accuracy: {avg_val_accuracy:.4f}")
    print("======================================")


Epoch 1/10
Average training loss: 1.8566
Average validation loss: 1.6234
Average validation accuracy: 0.3983
Epoch 2/10
Average training loss: 1.4948
Average validation loss: 1.4242
Average validation accuracy: 0.5043
Epoch 3/10
Average training loss: 1.1483
Average validation loss: 1.3578
Average validation accuracy: 0.5381


KeyboardInterrupt: ignored

In [23]:
# 입력할 문장
text = "고마워"

# 입력된 문장을 토큰화
inputs = tokenizer(text, return_tensors="pt")

# GPU 사용 여부에 따라 입력을 적절한 장치로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델에 입력값을 넣고 예측
outputs = model(**inputs)
logits = outputs.logits

# 예측 결과를 softmax 함수를 이용하여 확률값으로 변환
probs = torch.nn.functional.softmax(logits, dim=-1)

# 확률값 중에서 가장 큰 값의 인덱스를 예측 클래스로 사용
pred_class = torch.argmax(probs, dim=-1)

# 확률값 및 예측 클래스 출력
class_dict = {0:'행복', 1:'슬픔', 2:'공포', 3:'중립', 4:'분노', 5:'혐오', 6:'놀람'}
print(probs.tolist())
print("예측 클래스:", class_dict[pred_class.item()])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[0.2944757342338562, 0.16595087945461273, 0.10767258703708649, 0.19245122373104095, 0.09722556173801422, 0.06834037601947784, 0.07388365268707275]]
예측 클래스: 행복


In [64]:
from sklearn.model_selection import KFold

kf_split_num = 5
kf = KFold(n_splits=kf_split_num, shuffle=True, random_state=42)

for fold, (train_idx,val_idx) in enumerate(kf.split(range(len(train_data)))):
    
    # fold로 나눠진 인덱스로 train_data 한번 더 나누기
    k_train_data = TensorDataset(train_inputs[train_idx], train_masks[train_idx], train_labels[train_idx])
    k_valid_data = TensorDataset(train_inputs[val_idx], train_masks[val_idx], train_labels[val_idx])

    k_train_sampler = RandomSampler(k_train_data)
    k_train_dataloader = DataLoader(k_train_data, sampler=k_train_sampler, batch_size=batch_size)

    k_validation_sampler = SequentialSampler(k_valid_data)
    k_validation_dataloader = DataLoader(k_valid_data, sampler=k_validation_sampler, batch_size=batch_size)

    # 모델 학습
    num_epochs = 5
    for epoch in range(num_epochs):
        # 학습 모드로 변경
        model.train()
        
        # 손실값 초기화
        total_loss = 0
        
        # 배치별로 학습
        for step, batch in enumerate(k_train_dataloader):
            # 배치 데이터 GPU로 이동
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # 그래디언트 초기화
            optimizer.zero_grad()
            
            # forward 수행
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs[0]
            
            # 손실값 누적
            total_loss += loss.item()
            
            # backward 수행 및 그래디언트 클리핑
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # 옵티마이저 업데이트
            optimizer.step()
            
        # 평균 손실값 계산
        avg_train_loss = total_loss / len(k_train_dataloader)
        
        # 검증 모드로 변경
        model.eval()
        
        # 검증 데이터셋을 이용한 검증
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0
        
        for batch in k_validation_dataloader:
            # 배치 데이터 GPU로 이동
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # 그래디언트 계산하지 않음
            with torch.no_grad():
                outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                loss = outputs[0]
                logits = outputs[1]
                
            # 검증 손실값 누적
            total_eval_loss += loss.item()
            
            # 예측값 추출
            preds = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            
            # 정확도 계산
            total_eval_accuracy += flat_accuracy(preds, label_ids)
        
        # 평균 검증 손실값 및 정확도 계산
        avg_val_accuracy = total_eval_accuracy / len(k_validation_dataloader)
        avg_val_loss = total_eval_loss / len(k_validation_dataloader)
        
        # 결과 출력
        print(f"Fold {fold+1}/{kf_split_num}")
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Average training loss: {avg_train_loss:.4f}")
        print(f"Average validation loss: {avg_val_loss:.4f}")
        print(f"Average validation accuracy: {avg_val_accuracy:.4f}")
        print("======================================")

Fold 1/5
Epoch 1/5
Average training loss: 0.0561
Average validation loss: 0.0397
Average validation accuracy: 0.9878
Fold 1/5
Epoch 2/5
Average training loss: 0.0769
Average validation loss: 0.0451
Average validation accuracy: 0.9878
Fold 1/5
Epoch 3/5
Average training loss: 0.0644
Average validation loss: 0.0683
Average validation accuracy: 0.9792
Fold 1/5
Epoch 4/5
Average training loss: 0.0543
Average validation loss: 0.0489
Average validation accuracy: 0.9861
Fold 1/5
Epoch 5/5
Average training loss: 0.0451
Average validation loss: 0.0593
Average validation accuracy: 0.9844
Fold 2/5
Epoch 1/5
Average training loss: 0.0705
Average validation loss: 0.0659
Average validation accuracy: 0.9861
Fold 2/5
Epoch 2/5
Average training loss: 0.0519
Average validation loss: 0.0631
Average validation accuracy: 0.9844
Fold 2/5
Epoch 3/5
Average training loss: 0.0320
Average validation loss: 0.0608
Average validation accuracy: 0.9878
Fold 2/5
Epoch 4/5
Average training loss: 0.0630
Average validat